In [2]:
import os
import numpy as np
import cv2
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# data loading & stratified split
def load_data(image_dir, img_size=(150, 150)):
    images, labels = [], []
    for label in ["NORMAL", "PNEUMONIA"]:
        folder = os.path.join('/content/drive/MyDrive/Colab Notebooks/chest_xray', label)
        for fname in os.listdir(folder):
            if not fname.lower().endswith(".jpeg"):
                continue
            img = cv2.imread(os.path.join(folder, fname))
            img = cv2.resize(img, img_size)
            images.append(img)
            labels.append(0 if label == "NORMAL" else 1)
    return np.array(images), np.array(labels)

image_dir = "./chest_xray"
images, labels = load_data(image_dir)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(sss.split(images, labels))
X_train, y_train = images[train_idx], labels[train_idx]
X_test,  y_test  = images[test_idx],  labels[test_idx]

In [ ]:
#data generators with normalization + augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.25,
    zoom_range=0.25,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow(X_train, y_train, batch_size=32)
val_gen   = test_datagen.flow(X_test,  y_test,  batch_size=32)

In [ ]:
#build model with transfer learning + batchnorm

base_model = MobileNetV2(input_shape=(150,150,3), include_top=False, weights='imagenet')
base_model.trainable = False  # freeze

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

<ipython-input-5-47a1f636280d>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(150,150,3), include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 150, 150,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 75, 75,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 75, 75,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 75, 75,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 75, 75,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 75, 75,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 75, 75,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 75, 75,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 75, 75,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 75, 75,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 75, 75,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 75, 75,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 77, 77,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 38, 38,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 38, 38,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 38, 38,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 38, 38,    │      2,304 │ block_1_depthwis

 Total params: 2,422,593 (9.24 MB)

 Trainable params: 164,353 (642.00 KB)

 Non-trainable params: 2,258,240 (8.61 MB)

In [ ]:
#callbacks

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
]


In [ ]:
#train

history = model.fit(
    train_gen,
    steps_per_epoch=len(X_train)//32,
    epochs=25,
    validation_data=val_gen,
    validation_steps=len(X_test)//32,
    callbacks=callbacks
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
146/146 ━━━━━━━━━━━━━━━━━━━━ 147s 948ms/step - accuracy: 0.7629 - loss: 0.5384 - val_accuracy: 0.7569 - val_loss: 0.5521 - learning_rate: 0.0010
Epoch 2/25
  1/146 ━━━━━━━━━━━━━━━━━━━━ 1:44 718ms/step - accuracy: 0.7812 - loss: 0.5632

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


146/146 ━━━━━━━━━━━━━━━━━━━━ 41s 279ms/step - accuracy: 0.7812 - loss: 0.5632 - val_accuracy: 0.7552 - val_loss: 0.5599 - learning_rate: 0.0010
Epoch 3/25
146/146 ━━━━━━━━━━━━━━━━━━━━ 155s 916ms/step - accuracy: 0.8935 - loss: 0.2629 - val_accuracy: 0.8394 - val_loss: 0.3990 - learning_rate: 0.0010
Epoch 4/25
146/146 ━━━━━━━━━━━━━━━━━━━━ 24s 164ms/step - accuracy: 0.8750 - loss: 0.2893 - val_accuracy: 0.8516 - val_loss: 0.3808 - learning_rate: 0.0010
Epoch 5/25
146/146 ━━━━━━━━━━━━━━━━━━━━ 132s 902ms/step - accuracy: 0.8902 - loss: 0.2651 - val_accuracy: 0.9080 - val_loss: 0.2451 - learning_rate: 0.0010
Epoch 6/25
146/146 ━━━━━━━━━━━━━━━━━━━━ 41s 280ms/step - accuracy: 0.8750 - loss: 0.2922 - val_accuracy: 0.9054 - val_loss: 0.2580 - learning_rate: 0.0010
Epoch 7/25
146/146 ━━━━━━━━━━━━━━━━━━━━ 203s 1s/step - accuracy: 0.8978 - loss: 0.2374 - val_accuracy: 0.8872 - val_loss: 0.2881 - learning_rate: 0.0010
Epoch 8/25
146/146 ━━━━━━━━━━━━━━━━━━━━ 43s 291ms/step - accuracy: 0.8125 - loss:

In [ ]:
# fine tune

base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])
history_fine = model.fit(
    train_gen,
    steps_per_epoch=len(X_train)//32,
    epochs=10,
    validation_data=val_gen,
    validation_steps=len(X_test)//32,
    callbacks=callbacks
)

Epoch 1/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 188s 1s/step - accuracy: 0.8752 - loss: 0.3242 - val_accuracy: 0.8724 - val_loss: 0.3912 - learning_rate: 1.0000e-04
Epoch 2/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 40s 273ms/step - accuracy: 0.8125 - loss: 0.6111 - val_accuracy: 0.8759 - val_loss: 0.3816 - learning_rate: 1.0000e-04
Epoch 3/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 259s 1s/step - accuracy: 0.9118 - loss: 0.2484 - val_accuracy: 0.9375 - val_loss: 0.1739 - learning_rate: 1.0000e-04
Epoch 4/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 22s 142ms/step - accuracy: 0.9688 - loss: 0.1254 - val_accuracy: 0.9392 - val_loss: 0.1685 - learning_rate: 1.0000e-04
Epoch 5/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.9160 - loss: 0.2243 - val_accuracy: 0.9184 - val_loss: 0.2184 - learning_rate: 1.0000e-04
Epoch 6/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 23s 157ms/step - accuracy: 0.9062 - loss: 0.2744 - val_accuracy: 0.9149 - val_loss: 0.2306 - learning_rate: 1.0000e-04
Epoch 7/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step

In [ ]:
# evaluate

X_test_norm = X_test.astype('float32') / 255.0
y_pred_prob = model.predict(X_test_norm).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["NORMAL","PNEUMONIA"]))

37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 642ms/step
Confusion Matrix:
[[294  23]
 [ 34 822]]

Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.90      0.93      0.91       317
   PNEUMONIA       0.97      0.96      0.97       856

    accuracy                           0.95      1173
   macro avg       0.93      0.94      0.94      1173
weighted avg       0.95      0.95      0.95      1173



In [ ]:
# save model 

model.save("pneumonia_detection_model.h5")
print("\nModel saved to pneumonia_detection_model.h5")


Model saved to pneumonia_detection_model.h5
